In [97]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2018-11-29 12:18:41--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.222, 91.189.89.223
Connecting to launchpad.net (launchpad.net)|91.189.89.222|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb [following]
--2018-11-29 12:18:42--  https://launchpadlibrarian.net/386846978/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpadlibrarian.net (launchpadlibrarian.net)... 91.189.89.228, 91.189.89.229
Connecting to launchpadlibrarian.net (launchpadlibrarian.net)|91.189.89.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1232624 (1.2M) [application/x-debian-package]
Saving to: ‘google-drive-ocamlfuse_

In [98]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.datasets import load_files
from sklearn.preprocessing import LabelBinarizer


In [0]:
def load_data(fold_path):
  data = load_files(fold_path)
  files = np.array(data['filenames'])
  return files

In [0]:
inp_path = 'drive/datasets/ISBI/images'
inp = load_data(inp_path)


In [0]:
lab_path = 'drive/datasets/ISBI/labels'
lab = load_data(lab_path)

In [0]:
from keras.models import Model
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, concatenate, Input
from keras.optimizers import *

In [0]:
def conv_layer(ip,filters,kernel_size,stride,padding):
  con = Conv2D(filters,kernel_size,strides=stride,padding=padding,activation='relu',kernel_initializer='random_uniform')(ip)
  bn = BatchNormalization(axis=-1)(con)
  conv = Conv2D(filters,kernel_size,strides=stride,padding=padding,activation='relu',kernel_initializer='random_uniform')(con)
  return conv

In [0]:
def u_net(input_size=(256,256,3),filt=16):
  inp = Input(shape=input_size)
  conv_1 = conv_layer(inp,filt,(3,3),(1,1),padding='same')
  pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)
  conv_2 = conv_layer(pool_1,filt*2,(3,3),(1,1),padding='same')
  pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)
  conv_3 = conv_layer(pool_2,filt*4,(3,3),(1,1),padding='same')
  pool_3 = MaxPooling2D(pool_size=(2,2))(conv_3)
  conv_4 = conv_layer(pool_3,filt*8,(3,3),(1,1),padding='same')
  pool_4 = MaxPooling2D(pool_size=(2,2))(conv_4)
  conv_5 = conv_layer(pool_4,filt*16,(3,3),(1,1),padding='same')
  pool_5 = MaxPooling2D(pool_size=(2,2))(conv_5)
  conv_6 = conv_layer(pool_5,filt*32,(3,3),(1,1),padding='same')
  pool_6 = MaxPooling2D(pool_size=(2,2))(conv_6)
  conv_7 = conv_layer(pool_6,filt*64,(3,3),(1,1),padding='same')
  up_8 = UpSampling2D(size=(2,2))(conv_7)
  conv_8 = Conv2D(filt*32,(3,3),strides=(1,1),padding='same',activation='relu')(up_8)
  merge_8 = concatenate([conv_8,conv_6],axis=-1)
  conv_8 = conv_layer(merge_8,filt*32,(3,3),(1,1),padding='same')
  up_9 = UpSampling2D(size=(2,2))(conv_8)
  conv_9 = Conv2D(filt*16,(3,3),strides=(1,1),padding='same',activation='relu')(up_9)
  merge_9 = concatenate([conv_9,conv_5],axis=-1)
  conv_9 = conv_layer(merge_9,filt*16,(3,3),(1,1),padding='same')
  up_10 = UpSampling2D(size=(2,2))(conv_9)
  conv_10 = Conv2D(filt*8,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='random_uniform')(up_10)
  merge_10 = concatenate([conv_10,conv_4],axis=-1)
  conv_10 = conv_layer(merge_10,filt*8,(3,3),(1,1),padding='same')
  up_11 = UpSampling2D(size=(2,2))(conv_10)
  conv_11 = Conv2D(filt*4,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='random_uniform')(up_11)
  merge_11 = concatenate([conv_11,conv_3],axis=-1)
  conv_11 = conv_layer(merge_11,filt*4,(3,3),(1,1),padding='same')
  up_12 = UpSampling2D(size=(2,2))(conv_11)
  conv_12 = Conv2D(filt*2,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='random_uniform')(up_12)
  merge_12 = concatenate([conv_12,conv_2],axis=-1)
  conv_12 = conv_layer(merge_12,filt*2,(3,3),(1,1),padding='same')
  up_13 = UpSampling2D(size=(2,2))(conv_12)
  conv_13 = Conv2D(filt,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='random_uniform')(up_13)
  merge_13 = concatenate([conv_13,conv_1],axis=-1)
  conv_13 = conv_layer(merge_13,filt,(3,3),(1,1),padding='same')
  conv_14 = Conv2D(2,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='random_uniform')(conv_13)
  conv_15 = Conv2D(1,(1,1),strides=(1,1),activation='sigmoid')(conv_14)
  
  model = Model(input=inp,output=conv_15)
  model.summary()
  return model
  

In [0]:
def get_im(im_file):
  im = cv2.imread(im_file)
  im = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
  im = cv2.resize(im,(256,256))
  im = im/255
  im = np.expand_dims(im,axis=2)
  #plt.imshow(im)
  #plt.show()
  return im

In [0]:
def get_imgs(fp):
  img_list = [get_im(im) for im in tqdm(fp)]
  img_list = np.stack(img_list,axis=0)
  return img_list

In [108]:
inp_img = get_imgs(inp)

100%|██████████| 30/30 [00:00<00:00, 106.46it/s]


In [109]:
op_img = get_imgs(lab)

100%|██████████| 30/30 [00:00<00:00, 163.52it/s]


In [110]:
mod = u_net(inp_img[0].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_165 (Conv2D)             (None, 256, 256, 16) 160         input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_166 (Conv2D)             (None, 256, 256, 16) 2320        conv2d_165[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_29 (MaxPooling2D) (None, 128, 128, 16) 0           conv2d_166[0][0]                 
__________________________________________________________________________________________________
conv2d_167

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
mod.compile(optimizer=SGD(lr=0.001,momentum=0.9),loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
mod.fit(inp_img,op_img,batch_size=None,steps_per_epoch=500,validation_steps=50,epochs=5,validation_split=0.2)

Train on 24 samples, validate on 6 samples
Epoch 1/5
500/500 [==============================] - 344s 689ms/step - loss: 0.5925 - acc: 0.7424 - val_loss: 0.5523 - val_acc: 0.7298
Epoch 2/5
500/500 [==============================] - 333s 667ms/step - loss: 0.5318 - acc: 0.7434 - val_loss: 0.5390 - val_acc: 0.7298
Epoch 3/5
432/500 [========================>.....] - ETA: 44s - loss: 0.5248 - acc: 0.7434

In [0]:
1